## 2021: Week 10 - Pokémon Hierarchies

I can hear the hesitancy already surrounding this challenge. "I know nothing about Pokémon, how can I participate?" Well, rest assured, you don't need to understand the data in order to work with it and complete this challenge. In fact, I believe this challenge could help with most hierarchical data. 

The aim of this challenge is to group Pokémon into Evolution Groups. Why? So that when we select any Pokémon, we can see all the information about what they evolved from, what they will evolve to etc. In the example below, Bulbasaur evolves into Ivysaur, who then evolves into Venusaur. Choose another Pokémon like Flareon and we can see it evolves from Eevee, who has multiple evolution possibilities.

How is this applicable to other hierarchical data? Well imagine that a Pokémon that has a null in the Evolves From field is similar to an employee at Work Level 1 in an organisation. Their manager would then sit in the Evolves To field. Our Evolution Group in this instance would be the Team the employee belongs to and it would give us the ability to type in any employee name and return information about which team they belong to and the structure of that team.

Sure, I could have chosen to design this challenge to be working with employees, but I believe it's useful to work with data outside of your comfort zone every now and then to prove to yourself that your prep skills are applicable no matter what the dataset. Plus it's just more fun to work with Pokémon!

### Input
We have 2 inputs this week:
1. Pokemon
![img](https://1.bp.blogspot.com/-6yOamFdTm1o/YCVAm0FbdtI/AAAAAAAAAvY/zQNLQy8hjAAiGj0Od3q1d1b27Mxnkml8ACLcBGAsYHQ/w400-h188/2021W10%2BInput1.png)

2. Evolution
![img2](https://1.bp.blogspot.com/-G9styqz2ZcI/YCVAaVD6E9I/AAAAAAAAAvU/O7pXXAZJsnQ3uXp2sgkSsUW8py1p6GieACLcBGAsYHQ/s320/2021W10%2BInput2.png)

### Requirment

- Input the data
- Our Pokémon dataset actually contains too many Pokémon: 
    - We're only interested in Pokémon up to Generation III, which is up to (and including) number 386
    - This means we're also not interested in mega evolutions so we can filter Pokémon whose name start with "Mega"
- Some Pokémon have more than one Type. We aren't interested in Types for this challenge so remove this field and ensure we have one row per Pokémon 
- Now we want to bring in information about what our Pokémon evolve to 
    - Warning!  In our Evolution dataset, we still have Pokémon beyond Gen III. You'll need to filter these out too, from both the evolved from and evolved to fields 
- Bring in information about what a Pokémon evolves from 
    - Ensure that we have all 386 of our Pokémon, with nulls if they don't have a pre-evolved form or if they don't evolve
- Finally, for Pokémon that have 3 evolutions, we want to know what the First Evolution is in their Evolution Group
- Some duplication may have occurred with all our joins, ensure no 2 rows are exactly the same
- Create a calculation for our Evolution Group
    - The Evolution Group will be named after the First Evolution e.g. in the above example, Bulbasaur is the name of the Evolution Group
- Output the data

### Output
![img3](https://1.bp.blogspot.com/-GXFwV1UpiX4/YCVijXraHSI/AAAAAAAAAvs/b9oxjFsWaucSvhHncE64YWynxWiBIe9tACLcBGAsYHQ/w640-h158/2021W%25600%2BOujt.png)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
data = pd.read_excel("./data/Pokemon Input.xlsx", sheet_name=["Pokemon", "Evolution"], dtype="object")

In [8]:
pokemon = data["Pokemon"].copy()
evolution = data["Evolution"].copy()

In [28]:
# Our Pokémon dataset actually contains too many Pokémon
# We're only interested in Pokémon up to Generation III, which is up to (and including) number 386
# This means we're also not interested in mega evolutions so we can filter Pokémon whose name start with "Mega"
pokemon = pokemon.rename(columns={"#": "Number"})
pokemon.Number = pokemon.Number.astype(float).astype(int)
pokemon = pokemon.sort_values(by="Number")
pokemon = pokemon.loc[(pokemon["Number"] <= 386)]

mega = pokemon["Name"].map(lambda x: x.startswith("Mega"))
pokemon = pokemon.loc[~mega, :]
pokemon

In [55]:
# Some Pokémon have more than one Type. 
# We aren't interested in Types for this challenge so remove this field and ensure we have one row per Pokémon
more_than_one_type = pokemon.groupby(["Name"])["Type"].nunique() != 1
more_than_one_type = more_than_one_type.reset_index()
more_than_one_type = more_than_one_type.loc[more_than_one_type["Type"] == True, "Name"]
pokemon[""]

2      Aerodactyl
3          Aggron
6         Altaria
8         Anorith
11        Ariados
          ...    
379        Wooper
382          Xatu
383         Yanma
385        Zapdos
387         Zubat
Name: Name, Length: 182, dtype: object

In [ ]:
pokemon.loc[more_than_one_type]

0      False
1      False
2      False
3      False
4      False
       ...  
615    False
616    False
617    False
618    False
619    False
Name: Name, Length: 620, dtype: bool